1. [Loading Data & Libraries](#1)
1. [Text Processing](#2)
1. [Build The Model](#3)
1. [Testing Model](#4)
1. [Model Training](#5)
1. [Generate Text](#6)
1. [Extra: Customized Training](#7)

<a id = "1"></a><br>
# Loading Data & Libraries

In [1]:
import tensorflow as tf

import numpy as np
import os
import time
# To ignore warnings
import warnings
warnings.filterwarnings('ignore')

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [2]:
file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 0s 0us/step


In [3]:
# Read, then decode for py2 compat.
text = open(file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


<a id = "2"></a><br>
# Text Processing

In [6]:
example = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [7]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [8]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

In [9]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [10]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [11]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [12]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [13]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [14]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i


In [15]:
seq_length = 100
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [16]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [17]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [18]:
dataset = sequences.map(split_input_target)

In [19]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


<a id = "3"></a><br>
# Build The Model

In [20]:
# Batch size
BATCH_SIZE = 30

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(30, 100), dtype=tf.int64, name=None), TensorSpec(shape=(30, 100), dtype=tf.int64, name=None))>

In [21]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 100

In [22]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [23]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [24]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(30, 100, 66) # (batch_size, sequence_length, vocab_size)


In [25]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  107400    
                                                                 
 dense (Dense)               multiple                  6666      
                                                                 
Total params: 130,962
Trainable params: 130,962
Non-trainable params: 0
_________________________________________________________________


<a id = "4"></a><br>
# Testing Model

In [26]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()
sampled_indices

array([17, 52, 61, 26, 51, 32, 55, 37, 27, 12, 34,  2, 39,  1, 30, 46, 56,
       28, 20, 30, 19, 28, 25, 53, 34, 39, 49, 20, 31, 61, 63, 15, 17, 46,
        0, 57, 20, 23, 59,  3,  9, 17, 40, 27, 44, 59, 46, 32, 25, 63, 64,
       23, 31,  3, 17, 44, 30, 45, 17, 64, 41, 27, 31, 43, 64, 43, 18, 21,
        7, 24,  9, 62,  1, 15, 49, 57, 17, 43,  6, 59, 52, 14, 14, 26, 49,
       34, 17, 63, 52, 41, 57,  0, 15, 21, 20, 64, 22, 24, 55,  9])

In [27]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'rted by my jealousies\nTo bloody thoughts and to revenge, I chose\nCamillo for the minister to poison\n'

Next Char Predictions:
 b"DmvMlSpXN;U Z\nQgqOGQFOLnUZjGRvxBDg[UNK]rGJt!.DaNetgSLxyJR!DeQfDybNRdydEH,K.w\nBjrDd'tmAAMjUDxmbr[UNK]BHGyIKp."


In [28]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [29]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (30, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.1856947, shape=(), dtype=float32)


In [30]:
tf.exp(example_batch_mean_loss).numpy()

65.73915

In [31]:
model.compile(optimizer='adam', loss=loss)

In [32]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

<a id = "5"></a><br>
# Model Training

In [33]:
EPOCHS = 20

In [34]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
368/368 [==============================] - 35s 83ms/step - loss: 2.5201
Epoch 2/20
368/368 [==============================] - 32s 83ms/step - loss: 1.9894
Epoch 3/20
368/368 [==============================] - 32s 83ms/step - loss: 1.8302
Epoch 4/20
368/368 [==============================] - 32s 83ms/step - loss: 1.7397
Epoch 5/20
368/368 [==============================] - 32s 83ms/step - loss: 1.6813
Epoch 6/20
368/368 [==============================] - 32s 83ms/step - loss: 1.6412
Epoch 7/20
368/368 [==============================] - 32s 83ms/step - loss: 1.6120
Epoch 8/20
368/368 [==============================] - 32s 83ms/step - loss: 1.5895
Epoch 9/20
368/368 [==============================] - 32s 84ms/step - loss: 1.5717
Epoch 10/20
368/368 [==============================] - 32s 84ms/step - loss: 1.5571
Epoch 11/20
368/368 [==============================] - 32s 83ms/step - loss: 1.5449
Epoch 12/20
368/368 [==============================] - 32s 84ms/step - loss: 1.5351
E

<a id = "6"></a><br>
# Generate Text

In [35]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [36]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [37]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:
Thus to way; for you hapse not therefore reverent, Camigd the find
the fathers.

ROMEO:
While with-he will prescumptoon.

LADY ANNE:
Well, if thou do. Anone,
A consues, but shall alook of him woild be rogute.
Married, selas farsh'd me! my soul and of concertion, 'Swerance were, by good Cist: I am sweet.

ANDOLISTUS:
AhQuess of you, say very fontorming wite can step him, he sleep,
Grengue
For bied,
Whether the love someted of yet his cowhian?

PRINCE EDLETIO:
O, become of the hows; are our green assuric, like their hasted littles being their to beatst it seater!

CLIENER:
To professiey, appose ye, and you see acquaint to
are she shall formers. But Juliet consul, believe thee
A breathe wnecs' heard arms; for God, the bain so wives us be the world: but thou may hast behold in Herry a fire you,
Made my guest.
This have tender the arives, bry angry dangerce?

Second Camice fellow:
The fuit him; yea therewax cusil.

EXEVELLA:
Agell for England,
Distround
And sweet foes are earth
As de

In [38]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b"ROMEO:\nBrother-rear'd queen doth no more to repant of this prince thou care?\n\nBAPTISTA:\nBoth let uncounded\nTo my good every coming my one, But was me not prayers.\n\nLEONTES:\nNo, not Josy.\n\nMENENIUS:\nWeld give entreatally, he spode angry by suncril o'ersituble's givenes faith\nto be pash to break thou face?\n\nSEBASTIAN:\nPasta't have done show if the oavens\nThey fall death's may: he could shame, sirnce,\nFor a hath his blot from them for the mores readless\nbe gentlemen farred, cheer into the good my greet my knough unpresaying'll heavent of Henry.\n\nJOHNSO:\nThat hour to your deedle, spite of your langman,\nThat of answer it dovel.\nWheir.\n\nQUEEN:\nWhat thou pardard's in'tly,\nOf men\nIn all.\n\nPOLIXS:\nOn, indeed\nye, mark upon old,\nTo the To these affect:\nBut thou cape\nTo harest to his intons myself.\nMay like the mut to from thim; take discklese you are been out of a spoke yet, burnds are head ten:\nNo roo their shall you gallen buss\nOn the prince, 

<a id = "7"></a><br>
# Extra: Customized Training

In [39]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

In [40]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [41]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [42]:
model.fit(dataset, epochs=1)

368/368 [==============================] - 35s 84ms/step - loss: 2.5158


In [43]:
EPOCHS = 10

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    mean.reset_states()
    for (batch_n, (inp, target)) in enumerate(dataset):
        logs = model.train_step([inp, target])
        mean.update_state(logs['loss'])

        if batch_n % 50 == 0:
            template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            print(template)

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
    print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 2.0751
Epoch 1 Batch 50 Loss 2.0612
Epoch 1 Batch 100 Loss 2.0480
Epoch 1 Batch 150 Loss 2.0740
Epoch 1 Batch 200 Loss 1.8980
Epoch 1 Batch 250 Loss 1.9498
Epoch 1 Batch 300 Loss 1.9100
Epoch 1 Batch 350 Loss 1.9060

Epoch 1 Loss: 1.9797
Time taken for 1 epoch 33.06 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 1.8784
Epoch 2 Batch 50 Loss 1.8797
Epoch 2 Batch 100 Loss 1.8273
Epoch 2 Batch 150 Loss 1.7824
Epoch 2 Batch 200 Loss 1.8335
Epoch 2 Batch 250 Loss 1.7618
Epoch 2 Batch 300 Loss 1.7721
Epoch 2 Batch 350 Loss 1.7972

Epoch 2 Loss: 1.8158
Time taken for 1 epoch 32.33 sec
________________________________________________________________________________
Epoch 3 Batch 0 Loss 1.7596
Epoch 3 Batch 50 Loss 1.7394
Epoch 3 Batch 100 Loss 1.7115
Epoch 3 Batch 150 Loss 1.7864
Epoch 3 Batch 200 Loss 1.7072
Epoch 3 Batch 250 Loss 1.7682
Epoch 3 Batch 300 Loss 1.7406
Epoch 3 Batch 350 Loss 1.7003

Epoch 3 Loss: 1.